###### Run the Simulation Functions
This notebook includes all the python packages needed.

In [1]:
%run Simulation_Functions.ipynb

Using matplotlib backend: module://ipympl.backend_nbagg
Populating the interactive namespace from numpy and matplotlib


# Dynamic Spectra with Beam Correction from Gain($\nu,\theta ,\phi$) near ALMA

All the healpy code is commented, the package isn't available for Windows.

In [2]:
path_E=r'C:\Users\msmm1\Documents\Universidad\Trabajo de Grado\Gain_EDGES'
path_M=r'C:\Users\msmm1\Documents\Universidad\Trabajo de Grado\Gain_MIST'
path_B=r'C:\Users\msmm1\Documents\Universidad\Trabajo de Grado\Gain_BOWTIE'

### Frequency range

In [3]:
f=arange(40,121,1)

### Get Galactic longitude (l) and latitude (b) from Haslam Map

The next lines were run on OSX and I extracted the output (Sky temperature and galactic coordinates) to the files $\textit{T_Hmap.txt}$ and $\textit{GalCoord_HaslamMap.txt}$

In [4]:
#T_Hmap = hp.read_map("lambda_haslam408_dsds.fits")  reads Haslam Map

T_Hmap=loadtxt('T_Hmap.txt') #Haslam Map Temperatures

In [5]:
#nside = hp.get_nside(T_Hmap)  #nside defines the resolution of the map
#npix = hp.nside2npix(nside)   #number of pixels of the map

#l, b = hp.pix2ang(nside, arange(npix), lonlat=True)   #(l, b) coordinates for each pixel 

gal_lb=loadtxt('GalCoord_HaslamMap.txt')   #(l, b) coordinates for each pixel 

gal_lb[:,0]   #gives the longitude of each pixel

array([ 45., 135., 225., ..., 135., 225., 315.])

In [6]:
g_coord=SkyCoord(gal_lb[:,0]*u.deg, gal_lb[:,1]*u.deg, frame='galactic')   #Each pixel's galactic coordinate with astropy

### Time and location for horizontal coordinates 

We will use the ALMA location and intervals of 6 minutes in a day for our simulation. The day of the simulation will be 2021-06-14.

In [7]:
Alma_loc = EarthLocation.of_site('ALMA')
Edges_loc = EarthLocation.of_site('Murchison Widefield Array')

start_t=Time('2021-06-14T00:00:00',format='isot')

times6=start_t + TimeDelta(6*60, format='sec')*arange(0,240,1) #Time array with dt=6 min

altaz6_Alma = AltAz(location=Alma_loc, obstime=times6)    #AltAz frame for each time of the day FOR ALMA
altaz6_Edges = AltAz(location=Edges_loc, obstime=times6)  #AltAz frame for each time of the day FOR EDGES

hours=arange(0,24,0.1)  #Same as times6, best for plotting

In [8]:
Alma_loc.geodetic, Edges_loc.geodetic

(GeodeticLocation(lon=<Longitude -67.755 deg>, lat=<Latitude -23.029 deg>, height=<Quantity 5000. m>),
 GeodeticLocation(lon=<Longitude 116.67081524 deg>, lat=<Latitude -26.7033194 deg>, height=<Quantity 377.83 m>))

In [9]:
start_t

<Time object: scale='utc' format='isot' value=2021-06-14T00:00:00.000>

### $G[\nu,\theta,\phi]$ Matrix

In [8]:
Gn_ISO=ones((len(f),91,360))
Gn_E=G_matrix(path_E,'\Gain_E_',len(f))
Gn_M=G_matrix(path_M,'\Gains-',len(f))
Gn_B=G_matrix(path_B,'\Gain_B_',len(f))

In [10]:
#Gn_M[0]       #First matrix (initial frequency)
#amax(Gn_M)    #Max value from all matrixes
#Gn_M[0,1,0]   #Matrix 0 (first frecuency), theta=0, phi=1

#TEST
#th=[0,1,2]
#ph=[0,3,0]

#Gn_M[0,th,ph]   #This is going to give us the Gain value for the first frequency and each pair of th and ph.

### Simulations of the Dynamic Spectra with the gain pattern $G[\nu,\theta,\phi]$  for EDGES

In [14]:
#T_ISO=DS(f,altaz6,2.5) #Not necessary, same for MINI MIST
TG_E=DS_G(f,altaz6_Edges,2.5,Gn_E)

100%|████████████████████████████████████████████████████████████████████████████████| 240/240 [47:15<00:00, 11.82s/it]


### Simulations of the Dynamic Spectra with the gain pattern $G[\nu,\theta,\phi]$ for other antennas

In [11]:
TG_M=DS_G(f,altaz6_Alma,2.5,Gn_M)    #MIST antenna
TG_B=DS_G(f,altaz6_Alma,2.5,Gn_B)    #BOWTIE antenna

100%|████████████████████████████████████████████████████████████████████████████████| 240/240 [31:18<00:00,  7.83s/it]


### Save simulations

In [17]:
savetxt(r'C:\Users\msmm1\Documents\Universidad\Trabajo de Grado\Plots MIST\DS+G EDGES.txt', TG_E)
#savetxt(r'C:\Users\msmm1\Documents\Universidad\Trabajo de Grado\Plots MIST\DS+G MIST.txt', TG_M)
#savetxt(r'C:\Users\msmm1\Documents\Universidad\Trabajo de Grado\Plots MIST\DS+G BOWTIE.txt', TG_B)

# Galactic longitude of the cenith

In [10]:
galcoord_Alma=GalCoord_Cenit(Alma_loc,times6)
galcoord_Edges=GalCoord_Cenit(Edges_loc,times6)

100%|████████████████████████████████████████████████████████████████████████████████| 240/240 [00:06<00:00, 39.39it/s]


In [12]:
gal_l_Alma=zeros(len(times6))
gal_l_Edges=zeros(len(times6))

for i in range(len(times6)):
    #print(galcoord_Alma[i].l.value)
    
    if (galcoord_Alma[i].l.value>=304.8): gal_l_Alma[i]=galcoord_Alma[i].l.value-360
    else: gal_l_Alma[i]=galcoord_Alma[i].l.value
        
    if (galcoord_Edges[i].l.value>=211): gal_l_Edges[i]=galcoord_Edges[i].l.value-360
    else: gal_l_Edges[i]=galcoord_Edges[i].l.value

304.8664168685936
306.6685771432588
308.46494826653833
310.2528315850778
312.02961273556275
313.7927856486419
315.5399739924176
317.2689496515401
318.97764795972074
320.66417952916885
322.3268386425227
323.9641082853469
325.57466199491824
327.1573627805324
328.71125943030336
330.2355805590244
331.72972677215193
333.19326132451954
334.62589964151067
336.0274980480502
337.3980420202768
338.73763423856593
340.04648268163163
341.3248889618244
342.57323706308097
343.7919826066975
344.981642737072
346.1427866902773
347.27602708299423
348.3820119381305
349.46141744595127
350.5149414457324
351.54329760225875
352.5472102436529
353.52740982158343
354.4846289514843
355.4195989886831
356.3330470958689
357.2256937579924
358.09825070204903
358.9514191811743
359.7858885848328
0.6023353395261614
1.4014220671317332
2.1837969708278733
2.95009342129166
3.7009297185765666
4.436909007629064
5.158619327885141
5.866633779677204
6.561510792344294
7.24379448093657
7.914015080280605
8.572689446862165
9.22032162

In [14]:
#savetxt('gal_l_cenit_Alma.txt',gal_l_Alma)
#savetxt('gal_l_cenit_Edges.txt',gal_l_Edges)

# Latitude simulations

In [10]:
Lats=arange(-90,95,5)*u.deg      #dlat=5 deg
Locs=EarthLocation.from_geodetic(0*u.deg,Lats)
#Locs.geodetic.lat.value
t_17h=Time('2021-06-14T17:00:00',format='isot')

In [15]:
TG_ISO_Lat=DS_G_Lat(f,t_17h,2.5,Gn_ISO,Locs)

100%|██████████████████████████████████████████████████████████████████████████████████| 37/37 [08:46<00:00, 14.22s/it]


In [13]:
TG_I_Lat=DS_G_Lat(f,t_17h,2.5,Gn_I,Locs)
TG_CA_Lat=DS_G_Lat(f,t_17h,2.5,Gn_CA,Locs)
TG_SA_Lat=DS_G_Lat(f,t_17h,2.5,Gn_SA,Locs)

100%|██████████████████████████████████████████████████████████████████████████████████| 37/37 [17:24<00:00, 28.23s/it]


In [14]:
savetxt(r'SimLatitudes ISOTROPIC.txt', TG_ISO_Lat)
#savetxt(r'SimLatitudes INITIAL.txt', TG_I_Lat)
#savetxt(r'SimLatitudes WITH WINGS.txt', TG_CA_Lat)
#savetxt(r'SimLatitudes WINGLESS.txt', TG_SA_Lat)

## 3D Dynamic Spectra Tests

In [26]:
def DS_G_3D(fs,t_array,alfa,G_n,Locs,name_dir):
    
    import os
    os.mkdir(r'./{}'.format(name_dir))
    
    DS_G_Lats=empty((Locs.size,len(t_array),len(fs)))
    
    for i in tqdm(range (Locs.size)):  
        
        altaz_t = AltAz(location=Locs[i], obstime=t_array)
        
        DS_G_Lat_i=DS_G(fs,altaz_t,alfa,G_n)
        
        savetxt(r'./{}/DSG_Lat_{}'.format(name_dir,Locs.geodetic.lat.value[i]), DS_G_Lat_i)
        
        DS_G_Lats[i]=DS_G_Lat_i

    return DS_G_Lats

In [27]:
DSG_3D_ISO=DS_G_3D(f,times6,2.5,Gn_ISO,Locs,'DSG_3D_ISOTROPIC')

 54%|█████████████████████████████████████████▋                                   | 130/240 [65:17:53<28:08, 15.35s/it]


  8%|██████▊                                                                          | 20/240 [04:38<50:42, 13.83s/it]


 63%|██████████████████████████████████████████████████▋                             | 152/240 [35:26<20:36, 14.06s/it]


  3%|█▉                                                                     | 1/37 [66:34:54<2396:56:42, 239694.52s/it]


KeyboardInterrupt: 

In [ ]:
import os
p='prueba'
os.mkdir(r'./{}'.format(p))

In [5]:
TG_E_old=loadtxt('DS+G EDGES.txt')

In [32]:
close('all')